# Face verification app

We build an app that will be able to verify the user's face by distinguishing it from other faces.

## Data dollection

Build a dataset by using local camera and taking snapshots. We need positive pairs of pictures (where both pictures belong to the same face) and negative pairs (for this, we have used the Labelled Faces in the Wild dataset with also some different faces from camera, and some frames wit no faces).

For this reason we create three directories, the first is the Anchor (first images in the pairs), the second is the Positive (second images in the pairs that match the anchors) and the third is the Negative directory (third images in the pairs that differentiate from the anchors).

__Import libraries:__

In [32]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
from tensorflow.keras.metrics import Precision, Recall
import tensorflow as tf 
from tensorflow.keras.utils import plot_model
import os
import cv2
import random
import numpy as np
from matplotlib import pyplot as plt
import uuid

__Verify GPU configured with Tensorflow:__

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
tf.test.is_gpu_available(cuda_only=False,min_cuda_compute_capability=None)

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.experimental.list_physical_devices()

__Create directories needed for our data:__

In [2]:
# Setup Paths
POS_PATH = os.path.join('data','positive')
NEG_PATH = os.path.join('data','negative')
ANC_PATH = os.path.join('data', 'anchor')
VER_PATH = os.path.join('application_data', 'verification_images')
INP_PATH = os.path.join('application_data', 'input_image')

In [ ]:
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)
os.makedirs(VER_PATH)
os.makedirs(INP_PATH)

__Uncompress the Labelled Faces in the Wild dataset and move them to /data/negative path:__

In [6]:
!tar -xf lfw.tgz

In [ ]:
# Move LFW images to data/negative
for directory in os.listdir('lfw'):
    for file in os.listdir(os.path.join('lfw', directory)):
        EX_PATH = os.path.join('lfw', directory, file)
        NEW_PATH = os.path.join(NEG_PATH, file)
        os.replace(EX_PATH, NEW_PATH)

__Use OpenCV to establish connection to the webcam and take pictures. The frame must be cropped to be 250x250. Keyboard actions specified in order to save the pictures in the corresponding folders and break:__

In [5]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Resize frame to 250x250 px
    frame = frame[190:190+250, 230:230+250, :]
    
    # Collect anchors
    if cv2.waitKey(1) & 0XFF == ord('a'):
        imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    # Collect positives
    if cv2.waitKey(1) & 0XFF == ord('p'):
        imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)     

    # Collect verifications
    if cv2.waitKey(1) & 0XFF == ord('v'):
        imgname = os.path.join(VER_PATH, '{}.jpg'.format(uuid.uuid1()))
        cv2.imwrite(imgname, frame)
    
    # Show image back to screen
    cv2.imshow('Image Collection', frame)
    
    # Breaking gracefully
    if cv2.waitKey(1) & 0XFF == ord('q'):
        break

# Release the webcam        
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

additionaly, after some model tests we decide to also captured some more negative samples.

a)photos from camera with no face at all (camera covered, wall )
b)photos from camera with other faces.

__Helper function to insert random noise (brightness, contrast, flip, jpeg_quality, saturation) to pictures that have been captured by the webcam, so that more samples are created and the model is able to generalize better:__

In [6]:
def data_augmentation(img):
    data = []
    for i in range(9):
        img = tf.image.stateless_random_brightness(img, max_delta=0.02, seed=(1,2))
        img = tf.image.stateless_random_contrast(img, lower=0.6, upper=1, seed=(1,3))
        img = tf.image.stateless_random_flip_left_right(img, seed=(np.random.randint(100),np.random.randint(100)))
        img = tf.image.stateless_random_jpeg_quality(img, min_jpeg_quality=90, max_jpeg_quality=100, seed=(np.random.randint(100), np.random.randint(100)))
        img = tf.image.stateless_random_saturation(img, lower=0.9, upper=1, seed=(np.random.randint(100),np.random.randint(100)))
        
        data.append(img)
        
    return data

In [7]:
def apply_data_augmentation(file_path):
    for file_name in os.listdir(os.path.join(file_path)):
        img_path = os.path.join(file_path, file_name)
        img = cv2.imread(img_path)
        augmented_images = data_augmentation(img)

        for image in augmented_images:
            cv2.imwrite(os.path.join(file_path, '{}.jpg'.format(uuid.uuid1())), image.numpy())

__Apply the data augmentation function to the whole Anchor and Positive directories:__

In [8]:
apply_data_augmentation(ANC_PATH)
apply_data_augmentation(POS_PATH)

__Take 580 sample file paths in a random shuffled manner from each directory:__

In [4]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'\*.jpg').take(580)
positive = tf.data.Dataset.list_files(POS_PATH+'\*.jpg').take(580)
negative = tf.data.Dataset.list_files(NEG_PATH+'\*.jpg').take(580)

__Define function to load images from filepaths, decode jpegs, scale pixel values to facilitate training and resize images to 105x105 (input dimension for our conv layer):__

In [5]:
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (105,105))
    img = img / 255.0
    return img

__Create labeled dataset with pairs of positives, negatives and labels (1,0)__

In [6]:
positives = tf.data.Dataset.zip((anchor,positive,tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor,negative,tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)

__function to preprocess the sample pairs:__

In [7]:
def preprocess_twin(input_img, validation_img, label):
    return(preprocess(input_img), preprocess(validation_img), label)

__Build dataloader pipeline to apply the preprocessing process, cache the dataset and shuffle in order to have both positive and negative pairs in train and test sets:__


In [8]:
data = data.map(preprocess_twin)
data = data.cache()
data = data.shuffle(buffer_size=2000)

__Split dataset to train and test at 0.7, create batches of 16 and prefetch to improve latency and throughput:__

In [9]:
train_data = data.take(round(len(data)*0.7))
train_data = train_data.batch(16)
train_data = train_data.prefetch(8)

In [10]:
test_data = data.skip(round(len(data)*0.7))
test_data = test_data.take(round(len(data)*0.3))
test_data = test_data.batch(16, drop_remainder =True)
test_data = test_data.prefetch(8)

__Build the embedding layer:__

In [11]:
def make_embedding():
    inp = Input(shape=(105,105,3), name='input_image')
    
    c1 = Conv2D(64, (10,10), activation='relu')(inp)
    m1 = MaxPooling2D(64, (2,2), padding='same')(c1)
    
    c2 = Conv2D(128, (7,7), activation='relu')(m1)
    m2 = MaxPooling2D(64, (2,2), padding='same')(c2)
    
    c3 = Conv2D(128, (4,4), activation='relu')(m2)
    m3 = MaxPooling2D(64, (2,2), padding='same')(c3)
    
    c4 = Conv2D(256, (4,4), activation='relu')(m3)
    f1 = Flatten()(c4)
    d1 = Dense(4096, activation='sigmoid')(f1)
    
    
    return Model(inputs=[inp], outputs=[d1], name='embedding')

In [12]:
embedding = make_embedding()

In [13]:
embedding.summary()

Model: "embedding"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_image (InputLayer)    [(None, 105, 105, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 96, 96, 64)        19264     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 128)       401536    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 21, 21, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 18, 18, 128)       26

__Define class of custom layer for L1 Distance between the embeddings:__

In [14]:
class L1Dist(Layer):
    def __init__(self, **kwargs):
        super().__init__()
        
    def call(self, input_embedding, validation_embedding):
        return tf.math.abs(input_embedding - validation_embedding)

__Build the final siamese network model so as to process the pairs of images, calculate the L1 distance and do the binary classification:__

In [15]:
def make_siamese_model():
    
    # Handle inputs
    input_image = Input(name='input_img', shape=(105,105,3))
    validation_image = Input(name='validation_img', shape=(105,105,3))
    
    
    siamese_layer = L1Dist()
    siamese_layer._name = 'distance'
    distances = siamese_layer(embedding(input_image), embedding(validation_image))
    
    # Classification Layer
    classifier = Dense(1, activation='sigmoid')(distances)
    
    return Model(inputs=[input_image, validation_image], outputs=classifier, name='siamese_network')

In [16]:
siamese_model = make_siamese_model()

__Define Loss and optimizer:__

In [17]:
binary_cross_loss = tf.losses.BinaryCrossentropy()
opt = tf.keras.optimizers.Adam(0.0001)

__Create checkpoints for model training:__

In [18]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(opt=opt, siamese_model=siamese_model)

__Define the training step process, by using the GradientTape.gradient and the apply_gradients(). Convert it to a graph representation for better performance:__

In [19]:
@tf.function
def train_step(batch):
    
    # Record all operations
    with tf.GradientTape() as tape:
        X = batch[:2]
        y = batch[2]
        
        yhat = siamese_model(X, training=True)
        loss = binary_cross_loss(y, yhat)
            
    # Calculate gradients
    grad = tape.gradient(loss, siamese_model.trainable_variables)
    
    # Calculate updated weights and apply to siamese model
    opt.apply_gradients(zip(grad, siamese_model.trainable_variables))
    
    return loss

__Training process to loop through the batches, perform the training step and accumulate statistics for the precision and recall for each epoch:__

In [20]:
def train(data, EPOCHS):
    # loop through epochs
    for epoch in range(1, EPOCHS+1):
        print('\n Epoch {}/{}'.format(epoch,EPOCHS))
        progbar = tf.keras.utils.Progbar(len(data))
        
        r = Recall()
        p = Precision()
        
        # Loop through each batch
        for idx, batch in enumerate(data):
            loss = train_step(batch)
            yhat = siamese_model.predict(batch[:2], verbose=0)
            r.update_state(batch[2], yhat)
            p.update_state(batch[2], yhat)
            progbar.update(idx+1)
        print(loss.numpy(), r.result().numpy(), p.result().numpy())
            
        if epoch % 10 == 0:
            checkpoint.save(file_prefix=checkpoint_prefix)

In [21]:
EPOCHS = 20

In [24]:
train(train_data, EPOCHS)


 Epoch 1/20
 2/51 [>.............................] - ETA: 16:03


KeyboardInterrupt



In [ ]:
#continue the model training on google colab due lack of gpu, check colab.ipynb

__Evaluate model on the test set:__

In [ ]:
r = Recall()
p = Precision()

for test_input, test_val, y_true in test_data.as_numpy_iterator():
   # yhat = siamese_model.predict([test_input, test_val], verbose=0)
    yhat = model.predict([test_input, test_val], verbose=0)
    r.update_state(y_true, yhat)
    p.update_state(y_true, yhat)

print(r.result().numpy(), p.result().numpy())

__Visualize the data:__

In [ ]:
plt.figure(figsize=(10,8))
plt.subplot(1,2,1)
plt.imshow(test_input[8])
plt.subplot(1,2,2)
plt.imshow(test_val[8])
plt.show()
print(y_true[8])

__Save the final model:__

In [25]:
siamese_model.save('siamesemodel_v3.h5')

In [22]:
model = tf.keras.models.load_model('siamesemodel_v3.h5', custom_objects={'L1Dist':L1Dist, 'BinaryCrossentropy':tf.losses.BinaryCrossentropy})

In [29]:
model.predict([test_input, test_val])

1/1 [==============================] - 2s 2s/step


array([[1.3862670e-07],
       [6.6421023e-08],
       [5.6948692e-01],
       [3.5669652e-07],
       [9.9996823e-01],
       [9.4013929e-01],
       [2.7302539e-07],
       [2.4127795e-07],
       [3.6209050e-07],
       [3.4748933e-01],
       [9.9994469e-01],
       [1.5882508e-06],
       [2.4431767e-01],
       [8.4498225e-08],
       [9.9999410e-01],
       [9.9725109e-01]], dtype=float32)

In [30]:
model.summary()

Model: "siamese_network"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_img (InputLayer)         [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 validaption_img (InputLayer)   [(None, 105, 105, 3  0           []                               
                                )]                                                                
                                                                                                  
 embedding (Functional)         (None, 4096)         38960448    ['input_img[0][0]',              
                                                                  'validaption_img[0

In [33]:
# Plot the model structure
plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


__Verification function - Compare the input image with each one of the verification images. If the number of images that have been predicted as positive (p>detection_threshold) is greater than the verification_threshold, then the person is verified:__

In [34]:
def verify(model, detection_threshold, verification_threshold):
    results = []
    for image in os.listdir(os.path.join('application_data', 'verification_images')):
        input_img = preprocess(os.path.join('application_data', 'input_image', 'input_image.jpg'))
        validation_img = preprocess(os.path.join('application_data', 'verification_images', image))
        
        result = model.predict(list(np.expand_dims([input_img, validation_img], axis=1)))
        results.append(result)
       
    detection = np.sum(np.array(results) > detection_threshold)
    verification = detection / len(os.listdir(os.path.join('application_data', 'verification_images')))
    verified = verification > verification_threshold
    
    return results, verified

__Use the webcam to capture a photo as an input for the verification:__

In [35]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    frame = frame[190:190+250, 230:230+250, :]
    
    cv2.imshow('Verification', frame)
    
    # Verification trigger
    if cv2.waitKey(10) & 0xFF == ord('v'):
        cv2.imwrite(os.path.join('application_data', 'input_image', 'input_image.jpg'), frame)
    
        results, verified = verify(model, 0.5, 0.5)
        print(verified)
        
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 216ms/step
True
1/1 [==============================] - 0s 221ms/step


1/1 [==============================] - 0s 271ms/step
True
